# TP Médiateur - Wrappers : Yasmine ZEMMOURI.

- Importation des librairies nécessaires :

In [1]:
import sqlite3
from sqlite3 import Error
import os
import pandas as pd

- Définition de fonctions :

In [2]:
#Création d'une connection à SQLite 
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("SQLite version sur la machine : ",sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            print("Connexion réussie.")
    return conn

In [3]:
#Création d'une fonction pour lire un fichier selon son extension (json, csv ou xlsx)
def read_file(file):
    extension = file.split('.')[-1].lower()

    if extension == 'csv':
        return pd.read_csv(file, delimiter=';')
    elif extension == 'json':
        return pd.read_json(file)
    elif extension == 'xlsx':
        return pd.read_excel(file)
    else:
        raise ValueError(f"Extension de fichier non prise en charge : {extension}")

In [4]:
#Définition d'une fonction wrapper qui prends en entrée la bdd, le fichier et le nom de la future table
def wrapper(file, connect, name):
    data = read_file(file)
    data = data.apply(lambda x: x.astype(str).str.lower() if x.dtype == 'object' else x)
    
    #Création d'une table selon le fichier et insertion des données via pandas.to_sql
    data.to_sql(name , connect, index=False, if_exists='append')

In [5]:
conn = create_connection(r"maDataBase.db")

SQLite version sur la machine :  2.6.0
Connexion réussie.


In [6]:
cur = conn.cursor()

## A- Wrappers :

In [7]:
wrapper('liste-des-expositions-ayant-recu-le-label-exposition-dinteret-national.csv',conn,'Expositions')

In [8]:
wrapper('frequentation-des-musees-de-france.csv',conn,'Frequentations')

In [9]:
wrapper('liste-officielle-museesdf-20220127-data.xlsx',conn,'Musees')

In [10]:
wrapper('bornes-irve.csv',conn,'Bornes')

In [11]:
wrapper('boulangeries-qui-proposent-la-baguette-des-franciliens.csv',conn,'Boulangeries')

In [12]:
wrapper('lile-de-france-fete-ses-restos.json',conn,'Restaurants')

In [13]:
# Obtenir la liste des tables crées (sources de données après passage par le wrapper)
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print("Tables : ", [table[0] for table in tables])

Tables :  ['Expositions', 'Frequentations', 'Musees', 'Bornes', 'Boulangeries', 'Restaurants']


## B- Médiateur :

--> Le médiateur choisi est SQLite : les vues sont crées selon les besoins de l'utilisateur.

### 1- Une vue avec la fréquentation des musées par région et année :

In [14]:
cur.execute("DROP VIEW IF EXISTS FrequentationRegionAnnee;")
cur.execute("""
        CREATE VIEW IF NOT EXISTS FrequentationRegionAnnee AS
        SELECT
            m.`Identifiant Muséofile`,
            f.nom_du_musee,
            f.regions,
            f.annee,
            m.'Département',
            m.'Commune', 
            m.'Adresse', 
            m.'Code Postal',
            f.payant,
            f.gratuit,
            f.total,
            m.'Téléphone', 
            m.'URL'
        FROM
            Frequentations f
        JOIN
            Musees m ON f.nom_du_musee = m.`Nom officiel du musée`;
        """)

### 2- Une vue avec le nombre maximal d'expositions par musée (+informations du musée) :

In [15]:
cur.execute("DROP VIEW IF EXISTS MaxExpositionsParMusee;")
cur.execute("""
        CREATE VIEW IF NOT EXISTS MaxExpositionsParMusee AS
        SELECT
            e.nom_du_musee,
            m."Région administrative",
            m."Département" ,
            m."Commune" ,
            m."Adresse" ,
            m."Lieu" ,
            m."Code Postal",
            m."Téléphone",
            m."URL",
            m."Latitude",
            m."Longitude",
            COUNT(*) AS max_expositions
        FROM
            Expositions e
        JOIN
            Musees m ON e.nom_du_musee = m."Nom officiel du musée"
        GROUP BY
            e.nom_du_musee,
            m."Région administrative",
            m."Département",
            m."Commune",
            m."Nom officiel du musée",
            m."Adresse",
            m."Lieu",
            m."Code Postal",
            m."Téléphone",
            m."URL",
            m."Latitude",
            m."Longitude",
            m."REF_Deps"
        ORDER BY
            max_expositions DESC;

        """)

### 3- Une vue qui contient les informations des bornes de recharge seulement de l'Ile de France :

### 4- Une vue qui contient les boulangeries franciliennes et les bornes à proximité :

In [16]:
cur.execute( "DROP VIEW IF EXISTS BornesIleFrance;")
cur.execute("""
            CREATE VIEW IF NOT EXISTS BornesIleFrance AS
            SELECT DISTINCT
                n_amenageur, 
                n_operateur, 
                n_enseigne, 
                id_station, 
                n_station,
                ad_station, 
                xlongitude, 
                ylatitude, 
                puiss_max,
                type_prise, 
                acces_recharge, 
                departement
            FROM
                bornes
            WHERE
                region = 'île-de-france';
    """)

### 4- Une vue qui contient les boulangeries franciliennes et les bornes à proximité :

In [17]:
cur.execute("DROP VIEW IF EXISTS BoulangeriesEtBornes;")
cur.execute( """
            CREATE VIEW IF NOT EXISTS BoulangeriesEtBornes AS
            SELECT
                b.nom AS boulangerie_nom,
                b.adresse AS boulangerie_adresse,
                b.ville AS boulangerie_ville,
                MAX(br.id_station) AS borne_id_station,
                MAX(br.n_station) AS borne_nom_station,
                MAX(br.ad_station) AS borne_adresse_station,
                MAX(br.type_prise) AS borne_type_prise,
                MAX(br.puiss_max) AS borne_puissance,
                MAX(ABS(b.latitude - br.ylatitude) + ABS(b.longitude - br.xlongitude)) AS distance_max_approximative
            FROM
                boulangeries b
            LEFT JOIN
                bornes br ON b.ville = br.departement
            WHERE
                ABS(b.latitude - br.ylatitude) + ABS(b.longitude - br.xlongitude) < 0.05
            GROUP BY
                b.nom, b.adresse, b.ville, b.departement;
                """)

    #regroupe les informations des boulangeries et des bornes de recharge pour voitures électriques dans la région d'Île-de-France, avec une condition de distance approximative inférieure à 0.05

### 5- Une vue qui contient les informations sur les restaurant dont la spécialité est la cuisine française :

In [18]:
cur.execute( "DROP VIEW IF EXISTS RestaurantsCuisineFrancaise;")
cur.execute( """
            CREATE VIEW IF NOT EXISTS RestaurantsCuisineFrancaise AS
            SELECT
                id,
                siret,
                nom_de_l_etablissement AS restaurant_nom,
                adresse_de_l_etablissement AS restaurant_adresse,
                commune,
                type_d_etablissement AS restaurant_type,
                telephone AS restaurant_telephone,
                site_web_de_l_etablissement AS restaurant_site_web,
                horaires_d_ouverture_et_de_fermeture_de_l_etablissement,
                accueil_des_personnes_a_mobilite_reduite,
                specialite_culinaire
            FROM
                restaurants
            WHERE
                specialite_culinaire LIKE '%cuisine française%';

                """)

In [19]:
# Obtenir la liste des vues après la création :
cur.execute("SELECT name FROM sqlite_master WHERE type='view';")
views = cur.fetchall()
print("Vues : ", [view[0] for view in views])

Vues :  ['FrequentationRegionAnnee', 'MaxExpositionsParMusee', 'BornesIleFrance', 'BoulangeriesEtBornes', 'RestaurantsCuisineFrancaise']


## C- Exemple de requêtes exécutées selon les besoins du scénario :

#### Quelles sont les musées ayant acceuillis plus de 5 expositions de la région Ile-de-France ?

In [20]:
sql_query = """
    SELECT
        *
    FROM
        MaxExpositionsParMusee
    WHERE
        max_expositions >= 5
        AND "Région administrative" = 'île-de-france'
    ORDER BY
        max_expositions DESC;
"""

# Exécution de la requête SQL et création d'un DataFrame
df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

,nom_du_musee,Région administrative,Département,Commune,Adresse,Lieu,Code Postal,Téléphone,URL,Latitude,Longitude,max_expositions
0,musée municipal,île-de-france,essonne,etampes,hôtel de ville,hôtel de ville,91150.0,01 69 92 69 12,www.mairie-etampes.fr/le-musee/,48.434307,2.161909,5
1,musée municipal,île-de-france,seine-et-marne,crécy-la-chapelle,s'adresser à la mairie,nan,77580.0,nan,nan,48.858133,2.908442,5
2,musée municipal,île-de-france,seine-et-marne,moret-sur-loing,place de samois,nan,77250.0,01 60 70 32 30,nan,48.373638,2.815046,5
3,musée municipal,île-de-france,seine-saint-denis,saint-ouen,12 rue albert-dhalenne,château de saint-ouen,93400.0,01 71 86 62 68,www.saint-ouen.fr/services-infos-pratiques/cul...,48.915848,2.330639,5
4,musée municipal,île-de-france,yvelines,saint-germain-en-laye,38 rue au pain,nan,78000.0,0134 51 05 12,www.saintgermainenlaye.fr/736/musee-municipal....,48.794403,2.127681,5


#### Quels sont les musées de Paris où on trouve plus de billets gratuits que payants ?

In [21]:
sql_query = """
            SELECT
                nom_du_musee,
                regions,
                Département,
                Commune,
                Adresse,
                Téléphone,
                URL,
                payant,
                gratuit,
                total
            FROM
                FrequentationRegionAnnee
            WHERE
                gratuit >= payant
                AND total > 1000000
                AND Département= 'paris'
            ORDER BY
                total DESC;
"""

df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

,nom_du_musee,regions,Département,Commune,Adresse,Téléphone,URL,payant,gratuit,total
0,musée du louvre,île-de-france,paris,paris (1er),"musée du louvre, 75058 paris cedex 01",01 40 20 50 50,www.louvre.fr/,1181523.0,1643516.0,2825039.0
1,musée du louvre,ile-de-france,paris,paris (1er),"musée du louvre, 75058 paris cedex 01",01 40 20 50 50,www.louvre.fr/,1222612.0,1474972.0,2697584.0
2,musée du quai branly - jacques chirac,ile-de-france,paris,paris,37 quai branly,01 56 61 70 00,www.quaibranly.fr/fr/,607313.0,673309.0,1280622.0
3,musée du quai branly - jacques chirac,ile-de-france,paris,paris,37 quai branly,01 56 61 70 00,www.quaibranly.fr/fr/,564215.0,697602.0,1261817.0
4,"petit palais, musée des beaux-arts de la ville...",ile-de-france,paris,paris,avenue winston-churchill,01 53 43 40 00,www.petitpalais.paris.fr/,397257.0,806553.0,1203810.0
...,...,...,...,...,...,...,...,...,...,...
260,musée de la chasse et de la nature,ile-de-france,paris,paris (3ème),62 rue des archives,01 53 01 92 40,www.chassenature.org/,0.0,0.0,NaN
261,musée hébert,ile-de-france,paris,paris (6ème),"85, rue du cherche-midi",01 42 22 23 82,nan,0.0,0.0,NaN
262,musée des monuments français,ile-de-france,paris,paris,1 place du trocadéro,01 58 51 52 00,www.citedelarchitecture.fr/fr/article/le-musee...,0.0,0.0,NaN
263,musée d'ennery,ile-de-france,paris,paris (16ème),"59, avenue foch",01 45 53 57 96,www.guimet.fr/,0.0,0.0,NaN


#### Quelles sont les bornes de Paris, où la puissance maximal est supérieur à 20 et le type de prise est t2-ef ?

In [22]:
sql_query = """
        SELECT
            n_amenageur, 
            n_operateur, 
            n_enseigne, 
            id_station, 
            n_station as nom_station,
            ad_station as adresse, 
            xlongitude, 
            ylatitude, 
            puiss_max,
            type_prise
        FROM
            BornesIleFrance
        WHERE
            puiss_max > 20
            AND departement = 'paris'
            AND type_prise='t2 - e/f'; 
"""

df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

Pas de données pour cette requête.


,n_amenageur,n_operateur,n_enseigne,id_station,nom_station,adresse,xlongitude,ylatitude,puiss_max,type_prise


#### Quelles sont les boulangeries à proximité de bornes dont le type de prise est t2-ef ?

In [23]:
sql_query = """
      SELECT
        boulangerie_nom,
        boulangerie_adresse,
        boulangerie_ville,
        borne_id_station,
        borne_nom_station,
        borne_adresse_station,
        borne_puissance
    FROM
        BoulangeriesEtBornes
    WHERE
        borne_type_prise = 't2-ef';
"""

df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

,boulangerie_nom,boulangerie_adresse,boulangerie_ville,borne_id_station,borne_nom_station,borne_adresse_station,borne_puissance
0,au gout du temps,"104, rue bobillot",paris,fr*w75*pvp*0223,paris | rue étienne marcel 52,"97 boulevard de port-royal, 75014 paris",22.0
1,aux deux anges,"23, rue daval",paris,fr*w75*pvp*0229,paris | rue étienne marcel 52,"98 rue de turenne, 75003 paris",22.0
2,aux peches normands,"9, rue du faubourg du temple",paris,fr*w75*pvp*0222,paris | rue étienne marcel 52,"98 rue de turenne, 75003 paris",22.0
3,eva,"3, place du 25 août 1944",paris,fr*x75*pvp*0001,paris | rue vulpian 24,"97 boulevard de port-royal, 75014 paris",22.0
4,la boulangerie de jeanne,"109, avenue ledru-rollin",paris,fr*w75*pvp*0229,paris | rue étienne marcel 52,"98 rue de turenne, 75003 paris",22.0
5,la gourmandine,"221, rue de tolbiac",paris,fr*w75*pvp*0223,paris | rue étienne marcel 52,"97 boulevard de port-royal, 75014 paris",22.0
6,la maison kerck,"76, rue saint-maur",paris,fr*w75*pvp*0227,paris | rue étienne marcel 52,"98 rue de turenne, 75003 paris",22.0
7,le castel blangeois,"162, avenue ledru-rollin",paris,fr*w75*pvp*0229,paris | rue étienne marcel 52,"98 rue de turenne, 75003 paris",22.0
8,le grillon,"20, rue du poteau",paris,fr*w75*pvp*0221,paris | rue étienne marcel 52,"91 rue du faubourg saint denis, 75010 paris",22.0
9,legendre tolbiac,"79, rue de tolbiac",paris,fr*w75*pvp*0229,paris | rue vulpian 24,"98 rue de turenne, 75003 paris",22.0


#### Quels sont les restaurant, sur Paris, qui peuvent accueillir des handicapés ?

In [24]:
sql_query = """
       SELECT
    restaurant_nom,
    restaurant_adresse,
    restaurant_telephone,
    restaurant_site_web,
    horaires_d_ouverture_et_de_fermeture_de_l_etablissement,
    restaurant_type
FROM
    RestaurantsCuisineFrancaise
WHERE
    commune = 'paris'
    AND accueil_des_personnes_a_mobilite_reduite = 'oui';
"""

df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

,restaurant_nom,restaurant_adresse,restaurant_telephone,restaurant_site_web,horaires_d_ouverture_et_de_fermeture_de_l_etablissement,restaurant_type
0,le vin qui danse,69 rue broca,143315277.0,https://vqd.fr/,12 h à 15h,restaurant
1,atelier ramey,23 rue ramey,142510478.0,http://www.atelier-ramey.com/,7/7 déjeuners et dîners,bistronomique
2,le christine,1 rue christine,140517164.0,https://www.restaurantlechristine.com/,lun-ven : 12h00-14h30 // 18h30-23h00\nsam-dim ...,gastronomique
3,le petit pontoise,9 rue de pontoise,143292520.0,http://www.lepetitpontoise.fr/,lundi au dimanche\nmidi et soir\n12h-14h30\n18...,bistrot
4,les fous de l'île,33 rue des deux ponts,143257667.0,https://www.lesfousdelile.com/,ouvert 7j/7j\nen semaine : 12h-15h // 18h-23h\...,bistronomique
5,le bossu,17 rue des deux ponts,142392402.0,https://www.lebossu.com/,none,bistronomique
6,le colvert bistrot,54 rue saint-andré des arts,142037367.0,https://lecolvertbistrot.com/,ouvert 7j/7j de 12h à 23h,bistronomique
7,les fous de l’île,33 rue des deux ponts,143257667.0,https://www.lesfousdelile.com/,10h00-23h00 en continue,bistronomique


#### Quels sont les musées de Paris que au moins 1000 personnes ont visité, et qui ont eu au minimum 2 expositions de 2021 ?

In [26]:
sql_query = """
     SELECT
        FRA.annee,
        FRA.nom_du_musee,
        FRA.regions,
        FRA.département,
        FRA.commune,
        FRA.total AS frequentation_totale,
        MEM.max_expositions
    FROM
        FrequentationRegionAnnee FRA
    LEFT JOIN
        MaxExpositionsParMusee MEM ON FRA.nom_du_musee = MEM.nom_du_musee
    WHERE
        FRA.total > 1000
        AND MEM.max_expositions >= 2
        AND FRA.département = 'paris'
        AND FRA.annee = '2021'
    ORDER BY
        FRA.annee DESC;
"""

df = pd.read_sql_query(sql_query, conn)
if df.empty:
    print("Pas de données pour cette requête.")
    
df

,annee,nom_du_musee,regions,Département,Commune,frequentation_totale,max_expositions
0,2021,musée d'art et d'histoire du judaïsme,île-de-france,paris,paris (3ème),88725.0,2


## Fermeture de la connection :

In [ ]:
conn.commit()
cur.close()

In [ ]:
conn.close()
os.remove("maDataBase.db") 